# Regrid LFRic data

In [ ]:
import warnings

warnings.filterwarnings("ignore")

from functools import partial
from pathlib import Path

import iris
import numpy as np
import paths
from aeolus.calc import last_n_day_mean, time_mean, zonal_mean
from aeolus.const import add_planet_conf_to_cubes, init_const
from aeolus.io import create_dummy_cube, load_data, save_cubelist
from aeolus.lfric import (
    add_um_height_coord,
    fix_time_coord,
    load_lfric_raw,
    simple_regrid_lfric,
    ugrid_spatial,
)
from esmf_regrid.experimental.unstructured_scheme import regrid_unstructured_to_unstructured
from aeolus.model import lfric
from common import SIMULATIONS, SPINUP_DAYS

import os

os.environ["PROJ_IGNORE_CELESTIAL_BODY"] = "YES"

In [ ]:
sim_label = "hab1_mod_c192p"
# const = init_const(planet, directory=paths.const)

time_prof = "inst_diag"
# time_prof = "averages"

In [ ]:
grid_cell_area = load_lfric_raw(
    paths.data_raw
    / SIMULATIONS[sim_label].work_name
    / "1"
    / f"run*"
    / "lfric_initial.nc",
).extract_cube("grid_cell_area")
grid_cell_area.attributes.pop("invalid_standard_name");
# cell_width = grid_cell_area**0.5
# cell_width.convert_units("km")
# cell_width.rename("grid_cell_width")
# cell_width.attributes.pop("invalid_standard_name");

In [ ]:
add_levs = partial(
    add_um_height_coord,
    path_to_levels_file=paths.vert / f"vertlevs_{SIMULATIONS[sim_label].vert_lev}",
)


def combi_callback(cube, field, filename):
    [
        fix_time_coord(cube, field, filename),
        add_levs(cube, field, filename),
    ]

In [ ]:
fnames = sorted(
    paths.data_raw.glob(
        str(
            Path(SIMULATIONS[sim_label].work_name)
            / "*"
            / "run_lfric_atm_*"
            / f"lfric_{time_prof}.nc"
        )
    ),
    key=lambda x: int(x.parent.parent.name),
)

fnames = [
    i
    for i in fnames
    if int(i.parent.parent.name) * SIMULATIONS[sim_label].days_per_job > SPINUP_DAYS
]

In [ ]:
dset_raw = load_lfric_raw(
    fnames,
    callback=combi_callback,
    drop_coord=["forecast_reference_time"],
)
for cube in dset_raw:
    if cube.units == "ms-1":
        cube.units = "m s-1"

for i, cube in enumerate(dset_raw):
    print(f"---|---|-{'-'*30}-|-{'-'*60}")
    print(f"{i:<2d} | {cube.ndim} | {cube.var_name:>30} | {cube.name():>60}")

## Unstructured -> unstructured

In [ ]:
tgt_sim_label = "hab1_mod_c192s10e"
tgt_cube = load_lfric_raw(
    paths.data_raw
    / SIMULATIONS[tgt_sim_label].work_name
    / "1"
    / f"run*"
    / "lfric_initial.nc",
).extract_cube("grid_cell_area")

In [ ]:
%%time
dset_regr = iris.cube.CubeList()
for src_cube in dset_raw:
    dset_regr.append(regrid_unstructured_to_unstructured(src_cube, tgt_cube))

In [ ]:
const = init_const(SIMULATIONS[sim_label].planet, directory=paths.const)
add_planet_conf_to_cubes(dset_regr, const=const)

In [ ]:
# Write the data to a netCDF file
outdir = paths.data_proc / SIMULATIONS[sim_label].work_name
outdir.mkdir(parents=True, exist_ok=True)
gl_attrs = {
    "name": SIMULATIONS[sim_label].work_name,
    "planet": SIMULATIONS[sim_label].planet,
    "processed": "True",
}
chunk_label = f"_{int(fnames[0].parent.parent.name):03d}"
chunk_label += f"-{int(fnames[-1].parent.parent.name):03d}"
fname_out = (
    outdir
    / f"{SIMULATIONS[sim_label].work_name}_{time_prof}_{chunk_label}_regr_to_{tgt_sim_label}.nc".lower()
)
save_cubelist(dset_regr, fname_out, **gl_attrs)
print(f"Saved to: {str(fname_out)}")

## Unstructured -> rectilinear

In [ ]:
cubes_to_regrid = time_mean(dset_raw, model=lfric)
# cubes_to_regrid = iris.cube.CubeList([i for i in dset_raw])
cubes_to_regrid.append(grid_cell_area)

In [ ]:
n_res = 512
tgt_cube = create_dummy_cube(n_res=n_res, pm180=True)

In [ ]:
%%time
dset_regr = simple_regrid_lfric(
    cubes_to_regrid,
    tgt_cube=tgt_cube,
    ref_cube_constr=lfric.thta if time_prof == "averages" else lfric.caf,
    interp_vertically=(time_prof == "averages"),
)

for i, cube in enumerate(dset_regr):
    print(f"---|---|-{'-'*30}-|-{'-'*60}")
    print(f"{i:<2d} | {cube.ndim} | {cube.var_name:>30} | {cube.name():>60}")

In [ ]:
const = init_const(SIMULATIONS[sim_label].planet, directory=paths.const)
add_planet_conf_to_cubes(dset_regr, const=const)

In [ ]:
# Write the data to a netCDF file
outdir = paths.data_proc / SIMULATIONS[sim_label].work_name
outdir.mkdir(parents=True, exist_ok=True)
gl_attrs = {
    "name": SIMULATIONS[sim_label].work_name,
    "planet": SIMULATIONS[sim_label].planet,
    "processed": "True",
}
chunk_label = f"_{int(fnames[0].parent.parent.name):03d}"
chunk_label += f"-{int(fnames[-1].parent.parent.name):03d}"
fname_out = (
    outdir
    / f"{SIMULATIONS[sim_label].work_name}_{time_prof}_{chunk_label}_regr_{n_res}.nc".lower()
)
save_cubelist(dset_regr, fname_out, **gl_attrs)
print(f"Saved to: {str(fname_out)}")